In [206]:
import pandas as pd
import numpy as np
import re
import sys
from os.path import join as pjoin

In [208]:
# point to files
asset_file = pjoin('data','goal2','PFDasset.csv')
trans_file = pjoin('data','goal2','PFDtrans.csv')
income_file = pjoin('data','goal2','PFDincome.csv')
gift_file = pjoin('data','goal1','PFDgift.csv')
honoraria_file = pjoin('data','goal1','PFDhonoraria.csv')
libaility_file = pjoin('data','goal1','PFDliability.csv')
psoition_file = pjoin('data','goal1','PFDposition.csv')
tarvel_file = pjoin('data','goal1','PFDtravel.csv')

# load files
asset = pd.read_csv(asset_file, low_memory=False)
trans = pd.read_csv(trans_file, low_memory=False)
income = pd.read_csv(income_file, low_memory=False)
gift = pd.read_csv(gift_file, low_memory=False)
honoraria = pd.read_csv(honoraria_file, low_memory=False)
liability = pd.read_csv(libaility_file, low_memory=False)
position = pd.read_csv(psoition_file, low_memory=False)
travel = pd.read_csv(tarvel_file, low_memory=False)

# remove NAs
asset.fillna('', inplace=True)
trans.fillna('', inplace=True)
income.fillna('',inplace=True)
gift.fillna('',inplace=True)
honoraria.fillna('',inplace=True)
liability.fillna('',inplace=True)
position.fillna('',inplace=True)
travel.fillna('',inplace=True)

In [209]:
cols = ['CID','Chamber','CalendarYear','RealCode','RealCode2','AssetSource','Asset4Date','TransactionType','GiftInfo',
        'Location']

In [210]:
def map_ultorg(row, schedule):
    if row['Ultorg'] != '':
        assetsource = row['Ultorg']
    elif row['OrgName'] != '':
        assetsource = row['OrgName']
    else:
        source = '{}Source'.format(schedule)
        assetsource = row[source]
    return assetsource

def add_cols(df):
    for c in cols:
        if c not in df.columns:
            df[c] = ''
    return df.copy()

In [211]:
# modify trans to get the desired columns
trans['AssetSource'] = np.vectorize(lambda x,y:'{} {}'.format(x,y).strip())(trans['Asset4Transacted'],trans['Asset4Descrip'])
def match_trans_type(row):
    if not re.match('^ +$',row['Asset4Sold']):
        ttype = 'Sold'
    elif not re.match('^ +$',row['Asset4Exchanged']):
        ttype = 'Exchanged'
    elif not re.match('^ +$',row['Asset4Purchased']):
        ttype = 'Purchased'
    else:
        ttype = 'UnspecifiedTransactionType'
    return ttype
# # trans['TransactionType'] = ''
# for c in cols:
#     if c not in trans.columns:
#         trans[c] = ''
trans['TransactionType'] = trans.apply(match_trans_type, axis=1)
trans = add_cols(trans)

In [212]:
gift['Asset4Date'] = gift['GiftDate']
gift['TransactionType'] = 'Gift'
gift['SenAB'] = ''
gift['CalendarYear'] = gift['CalenderYear']
gift['RealCode2'] = ''
gift['Location'] = gift['GiftLocation']
gift['AssetSource'] = gift.apply(lambda x: map_ultorg(x,'Gift'), axis=1)
# TODO: check for and address the duplicates in gift

In [213]:
income.shape
income.head()
income['Location'] = income['IncomeLocation']
income['TransactionType'] = 'income'
income['AssetSource'] = income.apply(lambda x: map_ultorg(x,'Income'), axis=1)
for c in cols:
    if c not in income.columns:
        income[c] = ''

In [214]:
liability.shape
# print(liability.head())
liability.tail()
liability['LiabilitySource'] = liability['Creditor']
liability['TransactionType'] = 'liability'
liability['CalendarYear'] = liability['CalenderYear']
liability['Location'] = liability['LiabilityDate']
# liability[cols]
# liability.head()
liability['LiabilityDateText'].value_counts()

months_dict = {'january':'01/01/', 'february':'02/01/', 'march':'03/01/',
               'april':'04/01/', 'may':'05/01/', 'june':'06/01/',
               'july':'07/01/', 'august':'08/01/', 'september':'09/01/',
               'october':'10/01/','november':'11/01/', 'december':'12/01/',
               'jan':'01/01/', 'feb':'02/01/', 'mar':'03/01/',
               'apr':'04/01/', 'may':'05/01/', 'jun':'06/01/',
               'jul':'07/01/', 'aug':'08/01/', 'sep':'09/01/', 'oct':'10/01/',
               'nov':'11/01/', 'dec':'12/01/', 'sept':'09/01'}
def extend_year(year):
    if int(year) < 75:
        return '20{}'.format(year)
    else:
        return '19{}'.format(year)

def do_dates(date):
    if date == '':
        return date
    if re.match('^[0-9]{4}$',date):
        return '01/01/{}'.format(date)
    if re.match('^[0-9]{2}$',date):
        if int(date) > 75:
            return '01/01/19{}'.format(date)
        else:
            return '01/01/20{}'.format(date)
    bads = ['Purch','Late','REVOLVING']
    if len(date.split(' ')) > 1 and not any(b in date for b in bads):
        date = date.split(' ')
        month = date[0].lower()
        return "{}{}".format(months_dict[month],date[1])
    m = re.match('^[0-9]{6}$',date)
    if m:
        month, day, year = date[2:4],date[4:],date[:2]
        if int(year) < 75:
            year = '20{}'.format(year)
        else:
            year = '19{}'.format(year)
        return '{}/{}/{}'.format(month,day, year)
    m = re.match('^[0-9]{1,2}/[0-9]{2,4}$',date)
    if m:
        date = date.split('/')
        month = date[0]
        year = date[1]
        if len(year) < 4:
            year = extend_year(year)
        if len(month) == 1:
            month = '0{}'.format(month)
        return '{}/01/{}'.format(month,year)
    m = re.match('^[0-9]{2}/[0-9]{2}/\d{4}$',date)
    if m:
        return date
    m = re.match('^\d{2}/\d{2}/\d{2}$',date)
    if m:
        return '{}20{}'.format(date[:-2],date[-2:])
    if 'REVOLVING' in date:
        return ''
    if 'Late' in date:
        return date[-4:]
    m = re.match('^(\d{1})/(\d{1})/(\d{2})$',date)
    if m:
        year = extend_year(m.group(3))
        return '0{}/0{}/{}'.format(m.group(2),m.group(1),year)
    # TODO: this ignores ranges and only takes the latest date
    if '-' in date:
        date = date.split('-')
        date = date[1]
        if len(date) == 2:
            date = extend_year(date)
        return '01/01/{}'.format(date)
    if 'to' in date:
        date = date.split('to')[-1].strip()
        m = re.match('^(\d)/(\d{4})$',date)
        month = '0{}'.format(m.group(1))
        return '{}/01/{}'.format(month,m.group(2))
    if date == '2003/05': 
        return '01/01/2005'
    if date == '2003/2009':
        return '01/01/2009'
holder = liability['LiabilityDateText'].apply(do_dates)
liability['AssetSource'] = liability.apply(lambda x: map_ultorg(x,'Liability'), axis=1)

liability['Asset4Date'] = liability['LiabilityDateText']
for c in cols:
    if c not in liability.columns:
        liability[c] = ''

In [215]:
honoraria.shape
honoraria.head()
honoraria['Asset4Date'] = honoraria['HonorariaDate']
honoraria['Location'] = honoraria['HonorariaSourceLocation']
honoraria['TransactionType'] = 'honoraria'
honoraria['AssetSource'] = honoraria.apply(lambda x: map_ultorg(x,'Honoraria'), axis=1)
for c in cols:
    if c not in honoraria.columns:
        honoraria[c] = ''

In [216]:
position.shape
position.tail()
def pos_map_ultorg(row):
    if row['Ultorg'] != '':
        assetsource = row['Ultorg']
    elif row['OrgName'] != '':
        assetsource = row['OrgName']
    else:
        source = 'PositionOrg'
        assetsource = row[source]
    return assetsource
position['TransactionType'] = position['PositionHeld']
# position['AssetSource'] = position.apply(pos_map_ultorg, axis=1)
position['Location'] = position['PositionOrgLoc']
position.tail()
position['Asset4Date'] = np.vectorize(lambda x,y:'{} - {}'.format(x,y))(position['PositionFromDate'],position["PositionToDate"])
position = add_cols(position)

In [217]:
travel.shape
travel['AssetSource'] = travel.apply(lambda x:map_ultorg(x,'Travel'), axis=1)
def good_dates(x,y):
    x = '{}/{}/{}'.format(x[-2:],x[2:4],x[:2])
    y = '{}/{}/{}'.format(y[-2:],y[2:4],y[:2])
    return '{} to {}'.format(x,y)
travel['Asset4Date'] = np.vectorize(good_dates)(travel['BeginDate'],travel['EndDate'])
travel['Location'] = np.vectorize(lambda x,y:'{} {}'.format(x,y))(travel['Dummy City'],travel['Dummy State'])
# travel[travel.columns.values[15:]]
travel['TransactionType'] = 'travel'
travel = add_cols(travel)

In [218]:
idx = asset.shape[0] - 2459
asset_long = asset.iloc[:idx].copy()
asset_short = asset.iloc[idx:].copy()
asset_long.shape, asset_short.shape
asset_long['AssetSouce'] = asset_long.apply(lambda x: map_ultorg(x,'Asset'), axis=1)
asset_long['TransactionType'] = 'asset'
asset_long['Asset4Date'] = asset_long['Date']
asset_long = add_cols(asset_long)
asset_short['AssetSource'] = asset_short['Ultorg']
asset_short['OrgName'] = asset_short['RealCode']
asset_short['RealCode'] = asset_short['AssetDescrip']
asset_short['AssetSouce'] = asset_short.apply(lambda x: map_ultorg(x,'Asset'), axis=1)
asset_short['Asset4Date'] = ''
asset_short['TransactionType'] = 'asset'
asset_short = add_cols(asset_short)

In [219]:
goal_1_PFD = pd.concat([trans[cols], gift[cols], income[cols], honoraria[cols], liability[cols], position[cols], 
                        travel[cols], asset_long[cols], asset_short[cols]], ignore_index=True)
goal1_file =  pjoin('data','goal1','Goal1_PFD_concat.csv')
goal_1_PFD.to_csv(goal1_file,index=False)
